In [ ]:
!pip uninstall matplotlib

In [ ]:
!pip install matplotlib

In [1]:
import pandas as pd
import requests
import json
import seaborn as sns

In [2]:
import matplotlib.pyplot as plt

In [3]:
mt150 = pd.read_csv('C:/Users/Aaron/Documents/ML_Udacity/PPP/All Data by State/All Data by State/150k plus/PPP Data 150k plus.csv')

In [4]:
print("Data Set: PPP More than $150k \nRows: {}\nCols: {}".format(mt150.shape[0], mt150.shape[1]))

Data Set: PPP More than $150k 
Rows: 661218
Cols: 16


In [5]:
mt150.head()

,LoanRange,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsRetained,DateApproved,Lender,CD
0,a $5-10 million,"ARCTIC SLOPE NATIVE ASSOCIATION, LTD.",7000 Uula St,BARROW,AK,99723.0,813920.0,Non-Profit Organization,Unanswered,Unanswered,Unanswered,Y,295.0,04/14/2020,"National Cooperative Bank, National Association",AK - 00
1,a $5-10 million,CRUZ CONSTRUCTION INC,7000 East Palmer Wasilla Hwy,PALMER,AK,99645.0,238190.0,Subchapter S Corporation,Unanswered,Unanswered,Unanswered,NaN,215.0,04/15/2020,First National Bank Alaska,AK - 00
2,a $5-10 million,"I. C. E. SERVICES, INC",2606 C Street,ANCHORAGE,AK,99503.0,722310.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,367.0,04/11/2020,KeyBank National Association,AK - 00
3,a $5-10 million,KATMAI HEALTH SERVICES LLC,"11001 O'MALLEY CENTRE DRIVE, SUITE 204",ANCHORAGE,AK,99515.0,621111.0,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,0.0,04/29/2020,Truist Bank d/b/a Branch Banking & Trust Co,AK - 00
4,a $5-10 million,MATANUSKA TELEPHONE ASSOCIATION,1740 S. CHUGACH ST,PALMER,AK,99645.0,517311.0,Cooperative,Unanswered,Unanswered,Unanswered,NaN,267.0,06/10/2020,CoBank ACB,AK - 00


In [6]:
# Creates a new column 'NAICSCode_int' which converts 'NAICSCode' into integer.

mt150['NAICSCode_int'] = mt150['NAICSCode'].fillna('0').astype('int')

In [7]:
def get_naics(naicscode):
    '''Requests NAICS api to retrieve code title.'''
    
    page = requests.get("http://api.naics.us/v0/q?year=2012&code={}".format(naicscode))
    naicspage = json.loads(page.content)
    try:
        title = naicspage['title']
    except:
        title = 'NAICS Not Found'
    return title

In [ ]:
## get_naics(mt150['NAICSCode_int'][0])

In [ ]:
# Retrieve all NAICS Titles

# mt150['NAICS_title'] = mt150['NAICSCode_int'].apply(lambda x: get_naics(x))

In [8]:
%%time
top50try = mt150['NAICSCode_int'][:50].apply(lambda x: get_naics(x))

Wall time: 5.86 s


In [9]:
print('Would take {} hours to retrieve all NAICS titles.'.format(((mt150.shape[0] / 50) * 5.29) /3600))

Would take 19.432462333333334 hours to retrieve all NAICS titles.


In [ ]:
# mt150.groupby('NAICSCode_int')['BusinessName'].nunique().sort_values(ascending=False)[:20]

In [10]:
# Create new frame with only unique NAICS codes.

uqNAICSCodes = pd.DataFrame(mt150['NAICSCode_int'].unique(), columns=['NAICSCode_int'])

In [11]:
%%time
# Retrieve titles for Unique NAICS codes.

uqNAICSCodes['titles'] = uqNAICSCodes['NAICSCode_int'].apply(lambda x: get_naics(x))

Wall time: 2min 42s


In [12]:
# Merge Unique titles into main frame. 

mt150 = mt150.merge(uqNAICSCodes, on='NAICSCode_int')

In [13]:
# Split LoanRange into two: LoanCategory and LoanRange

mt150[['LoanCategory', 'LoanRange']] = mt150['LoanRange'].str.split(' ', 1, expand=True)

In [ ]:
pd.DataFrame(mt150.groupby('titles')[['BusinessName', 'LoanCategory']].count().sort_values(by='BusinessName', ascending=False))

In [ ]:
catplot_frame = pd.DataFrame(mt150.groupby(['titles', 'LoanRange']).agg({'BusinessName': 'count'}))
catplot_frame.head()

In [ ]:
sns.catplot(x = "LoanRange", y = "BuinessName", col = "titles")

In [14]:
mt150.head()

,LoanRange,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsRetained,DateApproved,Lender,CD,NAICSCode_int,titles,LoanCategory
0,$5-10 million,"ARCTIC SLOPE NATIVE ASSOCIATION, LTD.",7000 Uula St,BARROW,AK,99723.0,813920.0,Non-Profit Organization,Unanswered,Unanswered,Unanswered,Y,295.0,04/14/2020,"National Cooperative Bank, National Association",AK - 00,813920,Professional Organizations,a
1,$1-2 million,"THE WASHINGTON COUNTY HEALTH CARE AUTHORITY, INC.",14600 SAINT STEPHENS AVE,CHATOM,AL,36518.0,813920.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,201.0,04/10/2020,"The First, A National Banking Association",AL - 01,813920,Professional Organizations,c
2,"$350,000-1 million",AMERICAN PHARMACY COOPERATIVE INC,5601 SHIRLEY PARK DR,BESSEMER,AL,35022.0,813920.0,Non-Profit Organization,Unanswered,Unanswered,Unanswered,Y,56.0,04/06/2020,ServisFirst Bank,AL - 06,813920,Professional Organizations,d
3,"$150,000-350,000",STRATEGIC SYSTEMS INC,3434 Central Pkwy SW,DECATUR,AL,35603.0,813920.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,37.0,04/10/2020,Renasant Bank,AL - 04,813920,Professional Organizations,e
4,$1-2 million,WORLDATWORK,14040 N Northsight Blvd,SCOTTSDALE,AZ,85260.0,813920.0,Non-Profit Organization,Unanswered,Unanswered,Unanswered,Y,86.0,04/06/2020,BMO Harris Bank National Association,AZ - 06,813920,Professional Organizations,c


In [ ]:
# mt150.loc[mt150['City'] == 'NEW ORLEANS']

In [ ]:
## Francisco recommends:
##    folium